<a href="https://colab.research.google.com/github/ElioRame/ProgrammingAssignment2/blob/master/PALS0039_Ex_2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![PALS0039 Logo](https://www.phon.ucl.ac.uk/courses/pals0039/images/pals0039logo.png)](https://www.phon.ucl.ac.uk/courses/pals0039/)

#Exercise 2.3 Classification task

In this exercise we train a model to classify vowels from their [formant frequencies](https://en.wikipedia.org/wiki/Formant).

The following code reads in and summarises a data set of vowel formant frequencies.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

df = pd.read_csv("https://www.phon.ucl.ac.uk/courses/pals0039/data/exercise_02/vowels.csv")

print(df)
print("----------------------------------------------------")
print(df.describe())

(a) Create three boxplots that show the differences in F1, F2, and HEIGHT between male and female samples.

Hint: You could use [`plt.subplots`](https://matplotlib.org/stable/gallery/subplots_axes_and_figures/subplots_demo.html) to position all the plots in a row or column. You could use the [`boxplot` method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.boxplot.html) of `DataFrame` to create each plot.

In [ ]:
#(a)
#ANSWER
fig, axs = plt.subplots(2,3, figsize = (7, 12))

female = df.loc[df["SEX"] == "female"]
print(female)
F1_female = female["F1"]
F2_female = female["F2"]
Height_female = female["HEIGHT"]

male = df.loc[df["SEX"] == "male"]
F1_male = male["F1"]
F2_male = male["F2"]
Height_male = male["HEIGHT"]

axs[0, 0].boxplot(F1_female)
axs[0, 0].set_title('F1 Female')


axs[0, 1].boxplot(F2_female)
axs[0, 1].set_title('F2 Female')


axs[0, 2].boxplot(Height_female)
axs[0, 2].set_title('Female Height')


axs[1, 0].boxplot(F1_male)
axs[1, 0].set_title('F1 Male')


axs[1, 1].boxplot(F2_male)
axs[1, 1].set_title('F2 Male')


axs[1, 2].boxplot(Height_male)
axs[1, 2].set_title('Male Height')




---
(b) This code plots an F1-F2 scatter plot in which different vowels are displayed in different colours. Run the code and then add comments to the code to describe what is happening in each step.


In [ ]:
#(b)
#
df["VOWEL"]=df.VOWEL.astype("category")
#extracts categories
#
df["VOWELIDX"]=df.VOWEL.cat.codes
#creates a new column with category codes to then feed to the colour argument in plt.scatter
#
def plot_formants(data, f1="F1", f2="F2", axis_ranges=[3000,500,1100,100]):
  plt.figure(figsize=(10,10))
  plt.scatter(data[f2], data[f1], c=data.VOWELIDX, cmap="tab10")
  if axis_ranges: plt.axis(axis_ranges)
  plt.xlabel("F2")
  plt.ylabel("F1")
  plt.grid()

plot_formants(df)
plt.show()
print(df["VOWELIDX"])

(c) Interpreting the above scatter plot: Will perfect classification be possible using these measurements (F1 and F2 in Hz)? Why?

In [ ]:
#(c)
#ANSWER
#No, becayse the different vowels superimpose too much and cannot really be separated by f1 and f2 alone

The code below randomly selects a small held-out test set (which is plotted). The remaining samples are defined as the training set.

(d) Is the test set fully representative of the task? Why?

In [ ]:
test_set = df.sample(frac=0.05, random_state=0)
print(test_set.describe())
#extracting 5% of data
train_set = df.drop(test_set.index)
print(train_set.describe())

plot_formants(test_set)
plt.show()

print(train_set, test_set)
#(d)
#ANSWER
#No, vowels are missing and the set is too sparse

(e) Use `sklearn` to train a [Nearest Neighbour Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html) on the training set. The inputs should be `F1` and `F2` and the output should be the `VOWEL`. Configure the classifier to use the 3 nearest neighbours.

Hint: You need to define the classifier then call the [`fit` method](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier.fit)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

#(e)
#ANSWER
vowel = KNeighborsClassifier(n_neighbors = 3)
vowel.fit(X= train_set[['F1','F2']], y = train_set["VOWEL"])

(f) Determine the classification accuracy of your classifier on the train and test sets.

Hint: You can use the [`score` method](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier.score) of the classifier.

In [ ]:
#(f)
#ANSWER
print("The train set accuracy is:", vowel.score(X = train_set[["F1", "F2"]], y = train_set[["VOWEL"]]), sep = "\t")
print("The test set accuracy is:", vowel.score(X = test_set[["F1", "F2"]], y = test_set[["VOWEL"]]), sep = "\t")

(g) The following code normalises the data using the [z-score](https://en.wikipedia.org/wiki/Standard_score) for each speaker individually.

Add comments to each code block to explain what is happening.

In [ ]:
#(g)
#
#means = df.groupby(['SPEAKER']).agg("mean")
#stds = df.groupby(['SPEAKER']).agg("std")
# for each speaker, calculate mean and standard deviation across all samples of that speaker

means = df.groupby(['SPEAKER']).mean(numeric_only=True)

stds = df.groupby(['SPEAKER']).std(numeric_only=True)

#convert to numpy
F1mean = means.F1[df.SPEAKER].to_numpy()
F1std = stds.F1[df.SPEAKER].to_numpy()
F2mean = means.F2[df.SPEAKER].to_numpy()
F2std = stds.F2[df.SPEAKER].to_numpy()

#normalise each speaker value of F1 and F2 to obtain a comon scale
df["F1norm"] = (df.F1 - F1mean) / F1std
df["F2norm"] = (df.F2 - F2mean) / F2std

#
print(df.describe())

#
plot_formants(df, f1="F1norm", f2="F2norm", axis_ranges=None)
plt.show()

#
test_set = df.sample(frac=0.05, random_state=0)
print(test_set.describe())
train_set = df.drop(test_set.index)
print(train_set.describe())

(h) Train and evaluate a new classifier, as before, using the normalised formant data. What was the effect on the classification accuracy? Why?

In [ ]:
#(h)
#ANSWER
norm_vowel = KNeighborsClassifier(n_neighbors = 3)
norm_vowel.fit(X= train_set[['F1norm','F2norm']], y = train_set["VOWEL"])

print("The train set accuracy is:", norm_vowel.score(X = train_set[["F1norm", "F2norm"]], y = train_set[["VOWEL"]]), sep = "\t")
print("The test set accuracy is:", norm_vowel.score(X = test_set[["F1norm", "F2norm"]], y = test_set[["VOWEL"]]), sep = "\t")

#by normalising the accuracy improved

(i) In this exercise we calculated the statistics for normalisation on all the data (train and test sets combined), is this problematic? What would the consequence be when calculating the generalisation error? When deploying this system, how would we perform this normalisation for a new (unseen) speaker?

In [ ]:
#(i)
#ANSWER
#can be problematic because it was normalised to test samples, generalisation to unknown samples could therefore yield lower accuracy and would need to preprocess test data to obtain similar speaker statistics to make sure a commn scale is kept

(j) In this exercise we did not make use of a validation set, was it necessary? Why?

In [ ]:
#(j)
#ANSWER
#The use of a validation set would have been necessary if we wanted to find better hyperparameters for the classifier
#   (e.g. it is possible that using a larger number of neighbours could result in better generalisation error)